In [116]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from split import *
from score import *
#from scipy.stats import chi2
from scipy import interpolate
%matplotlib inline 
# When I was implementing the algorithm Search_by_Triplet I see that PANDAS can do data selections
# more easily. The  query method gives us this possibility. 
# Not have sense to implement the orderring of data and similar things. 

In [117]:
def rho(x,y):
    return np.sqrt(x*x + y*y)
def r(x,y,z):
    return np.sqrt(x*x + y*y + z*z)
def theta(x,y,z):
    return np.arccos(z/r(x,y,z))
def phi(x,y):
    return np.arctan(y/x) 

In [118]:
#df['theta'] = df.apply(lambda row: theta(row['x'], row['y'], row['z'] ), axis=1)
#df['phi']   = df.apply(lambda row: phi  (row['x'], row['y']),   axis=1)

In [6]:
def reading_data(fraction):
    name = 'data/RAMPData55microns50psInner200microns50psOuter_train.txt' # To be modified for others. 
    data_fraction = fraction

    df = pd.DataFrame()
    df = pd.read_csv(name, sep=' ') # All data.
    df,_ = Split_frac(df, data_fraction)
    return df    

In [120]:
# DATAFRAME will be a global data. 
fraction = 0.004 
dphi     = 0.01         #this information is a parameter that have to be evaluated as a function of time. 

df_original = reading_data(fraction)  # where data is unmodified.
df = df_original        
df_search   = df_original 

In [121]:
def sortbyphi(df):
    '''Description:
    Sort each D_i increasingly accourding to phi
    '''
    z = np.sort(df['z'].unique())
    df['phi'] = np.arctan(df['x']/df['y']) 
    modules = [] 
    for layer_i in z[::-1] :
        tmp = df.query(f'z=={layer_i}')
        #tmp['phi'] = tmp.apply(lambda row: phi(row['x'], row['y']), axis=1)
        
        tmp = tmp.sort_values('phi', ascending=True)
        modules.append(tmp)
    return modules 

In [122]:
def findcandidatewindows(left_mod, mod, right_mod, dphi):
    '''Description: 
        Compute the first and last candidates(the window) according to acceptance range(dphi) for each hit.
        SUPPOSSING THAT ALL DATA ARE ORDERED ACCOURDING TO PHI. THIS PROCCESS WAS DONE Previously
        In case of add more information to the modules, one easily can add throught the iteration 
    '''
    # CONVETION:     
    # l_m m  r_m         
    # |   |   |             
    # |   |   |    phi max  
    # |   |   |    phi      
    # |   |   |    phi min 
    # |   |   |             
    # The module in question is the one in the center
    #left_mod, mod, right_mod = modules 
    #dphi = acceptance_range  
    
    
    # This algorithm no develop a binary search. After, an intersting enhacement woudl be develop a function
    # in shuch a way that return the values of the condition
    
    #tmp = [] # Here I put the hit values related to right_hit_max, right_hit_min, or left_hit_max, left_hit_min
    
    #condition = f' {phi} - {dphi} < phi < {phi} - {dphi}' # condition
    #tmp = mod.query(condition)                            # defining the possibles hits
    #tmp['hit_id'][tmp.index[0]]       # the first hit   
    #tmp['hit_id'][tmp.index[-1]]      # the last  hit                                                    

    
    right_hit_max = []
    right_hit_min = []
    # ITERATION OVER PHI for RIGHT                     # ???????????????????? Here there are problems if not found values.
    for phi_i in mod['phi']:  
        # GET HIT 
        left      = phi_i - dphi
        right     = phi_i + dphi
        condition = f'{left} <= phi <= {right}'
        tmp_df    = right_mod.query(condition)
        #Getting the minimum and maximum
        print('=====================')
        print(tmp_df)
        print('=====================')
        m = 1 #tmp_df['hit_id'][tmp_df.index[0]]    # minumum hit
        M = 1000 #tmp_df['hit_id'][tmp_df.index[-1]]   # maximum hit
        right_hit_min.append(m)
        right_hit_max.append(M)

    left_hit_max = []
    left_hit_min = []
    # ITERATION OVER PHI FOR LEFT
    for phi_i in mod['phi']:  
        # GET HIT 
        left      = phi_i - dphi
        right     = phi_i + dphi                                                                                                             
        condition = f'{left} <= phi <= {right}'
        tmp_df = right_mod.query(condition)
        #Getting the minimum and maximum
        m = 1 #tmp_df['hit_id'][tmp_df.index[0]]    # minumum hit
        M = 1000 
        #m = tmp_df['hit_id'][tmp_df.index[0]]    # minumum hit
        #M = tmp_df['hit_id'][tmp_df.index[-1]]   # maximum hit
        left_hit_min.append(m)
        left_hit_max.append(M)
    
    # At the end of the algorithm we add the values of the left and right modules

    mod["right_hit_max"] = right_hit_max
    mod["right_hit_min"] = right_hit_min
    
    mod["left_hit_max"]  = left_hit_max
    mod["left_hit_min"]  = left_hit_min
    
    # With the help of two modules(left and riht)
    return mod

In [123]:
def trackseeding(mod, left_mod, right_mod):
    

    # mod_i are dataframes.
    # mod have the number of hits of max and min phi values. 
    # mod[] mod[]
    
    ''' 
    mod["right_hit_max"] = right_hit_max
    mod["right_hit_min"] = right_hit_min
    
    mod["left_hit_max"]  = left_hit_max
    mod["left_hit_min"]  = left_hit_min
    ''' 
    
    '''Description: 
        Checks the preceding and following modules for compatible hits using the above results.
        
        All triplets in the search window are fitted and compared.
        
        and the best one is kept as a track seed.
        
        stores its best found triplet
        Finding triplets is ap- plied in first instance to the modules
        that are further apart from the collision point
        Each triplet is the seed of a forming track
    '''
    #Necessary functions.
    def fit(triplet):
    
        print("*************triplet************************/n********************************")
        print(triplet)
        print("*************************************/n********************************")
        # triplet has the next form 
        # triplet = [[phi,z], [phi,z], [phi,z]]
        # or
        # triplet = (left_hit, center_hit, right_hit)
        #triplet0, triplet1, triplet2 = triplet[0], triplet[1], triplet[2]
        #phi_data  = [ (left_mod.query(f"   hit_id == {triplet0}  ")['phi'].values) ] 
        #phi_data += [ (mod.query(f"        hit_id == {triplet1}  ")['phi'].values) ] 
        #phi_data += [ (right_mod.query(f"  hit_id == {triplet2}  ")['phi'].values) ] 
                                                              
        #z_data  = [ (left_mod.query(f"   hit_id == {triplet0}  ")['z'].values) ]    
        #z_data  = [ (mod.query(f"        hit_id == {triplet1}  ")['z'].values) ]    
        #z_data += [ (right_mod.query(f"  hit_id == {triplet2}  ")['z'].values) ]    
        
        phi_data = [ df.query(f'hit_id == {827}')['phi'] for hit in triplet ]
        z_data   = [ df.query(f'hit_id == {827}')['z'] for hit in triplet ]
        phi_data = [ hit.values[0] for hit in phi_data ]
        z_data   = [ hit.values[0] for hit in z_data ]

        print(phi_data)
        print(z_data)
        
        
        print("[][][][][][][]][PHIDATA")
        print(phi_data)
        print("[][][][][][][]][PHIDATA")
        print(z_data)
        print("[][][][][][][]][][][][][]")
        
        # Kind of fit: Linear
        fitting = np.polyfit(phi_data, z_data, 1)
        
        chiSquared = np.sum((np.polyval(fitting, z_data) - phi_data)**2) #This part of the code I've used
        # from the chisquare tracking.ipynb
        # Finally I am returning the value of chi2
        return chiSquared
    
    df_triplets = pd.DataFrame(columns = ['lef_hit', 'hit', 'right_hit', 'chi2'])    
    
    for part in mod.iterrows():
        part = (part[1])  # IMPORTANT
        
        # hit_id
        # This process is neccessary 
        r_hit_max, r_hit_min = part["right_hit_max"], part["right_hit_min"]  
        l_hit_max, l_hit_min = part["left_hit_max"],  part["left_hit_min" ]  
        
        r_phi_max = right_mod.query(f"hit_id == {r_hit_max}")
        r_phi_min = right_mod.query(f"hit_id == {r_hit_min}")
        
        l_phi_max = left_mod.query(f"hit_id == {l_hit_max}")  
        l_phi_min = left_mod.query(f"hit_id == {l_hit_min}")  
        
        
        
        #Forming all Triplets. 
        print("##############tmp_left#############")
        tmp_left = df.query(f" {l_hit_min} <= phi <= {l_hit_max}")
        print(tmp_left)
        print("###################################")                
        for L in tmp_left['phi'].values: 
            tmp_right = df.query(f" {r_hit_min} <= phi <= {r_hit_max}")
            for R in tmp_right['phi'].values:
                
                print("------------------------The values of L R---are:-----------------------")
                print(L, R)
                print("--------------------------------------------------")
                # All information it is found on the hit values.
                hit_center = int(part["hit_id"])             
                hit_left = int(tmp_left.query(f" phi == {L}")['hit_id'].values)       
                hit_right = int(tmp_right.query(f" phi == {R}")['hit_id'].values)   
                
                print("------------------------The values of hits left and right ---are:-----------------------")
                print(hit_left, hit_center, hit_right)
                print("--------------------------------------------------")
                # With this data we have built the triplets. 
                triplets = [hit_left, hit_center, hit_right]
                # This a lost of memory. I mean that call by hits and not by values is a lost 
                # of memory.
                chi2 = fit(triplets)
                # Finally we append the values of the data to a df_triplets
                print("PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST")
                print(list(triplets)+ [chi2])
                print("PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST")
                df_triplets = df_triplets.append(list(triplets)+[chi2])

                
    # Up to this point it is necessary to have the values of df_triplets complete
    # Then the algorithm should continue to get the best choices according to the values
    # of chi2.  
                   
    def best_choice(df_triplets):
        #df_triplets have the next structure. 
        #[hl, h, hr, chi2]
        #So, I have to choose the best value
        tracks = [] 
        #tracks have the next structure:
        # [[1,2,3], [3,4,5,6,2,4,5], [3,3,4,6,78,4], [1,3,4]]
        # But at this step only we need to have the information of three values.
        # where all information are hits.
        # I have to do an iteration over the values of h
        # it means that
        for hit in df_triplets :
            tmp = df_triplets.query(f"hit == {hit}")
            chi2values = tmp['chi2'].values
            i = chi2values.idxmin() # Here we choice the values.
            #tracks.append((tmp.loc[i]).values) 
            tracks.insert(0, (tmp.loc[i]).values) # 
        return tracks

    print('????? WHAT IS TRACKS????')
    print(tracks)
    print('?????????')
    tracks = best_choice(df_triplets)
    return tracks

In [124]:
def track_forwarding(tracks, work_module, dphi):
    #Notation:
    # x0, y0, z0 is the EXTRAPOLATED track.               
    # X,  Y,  Z  is the last track on previous module.   
    # x,  y,  z  is the tracks on a window.                                                                 
    #Searching tracks on phi_e - dphi < phi < d that minimize the extrapolated function.

    #r0 = np.array([x0, y0, z0] )
    #r  = np.array([x, y, 1] )
    #R  = np.array([X,  Y,  Z ] )

    def mod(r):
        return np.sqrt(np.sum(r*r))

    def ext_func(r0, r1, r):
        # r0, r1, r are arrays
        dx2_plus_dy2 = mod(r0-r)  # distance between hits on the working module.
        dz2          = mod(r1-r0) # distance between the last two modules.                              
        return dx2_plus_dy2/dz2   
    
    new_tracks = [] 
    z_e = float(work_module['z'].unique()) #z_position of work_module

    for track in tracks:
        x_e = float(track['x'].unique()) #y_position of work_module
        y_e = float(track['y'].unique()) #x_position of work_module
        R = np.array(track[0]) 
        #work_module <- extrapolate_track on the working module
        data = []
        for hit in track[0:2] :
            data.append(tuple((df.query(f'hit_id == {hit}')[['phi', 'z']]).values[0]))
            
        z_data    = zip(*data)
        phi_data  = zip(*data) 
        #EXTRAPOLATED SEGMENT FUNCTION  
        ext_seg = interpolate.interp1d(z_data, phi_data, fill_value = "extrapolate" )

        #Open a Window centered on phi_e: 
        phi_e = ext_seg(z_e) 
        df_work_module_window = df_work_module.query(f" phi_e - dphi <= phi <= phi_e + dphi ") # I do not know how much time this line take 
        
        #This definition will be done after the loop.
        #df_candidates = pd.DataFrame(columns=["hit_id", "ext_fun"]) # This dataframe have to be reviwed
        
        
        hit_left = track[0]   
        R  = df.query(f'hit_id == {hit_left}')[['x','y','z']].values[0] # this value would have to change on each track
        r0 = np.array([x_e, y_e ,z_e ])
        
        
        tmp_candidates = []
        for index, row in df_work_module_window.iterrows(): # here the index is not important
            # Here I only need to have the information of position.
            r      =  row[['x', 'y', 'z']] # this value is variable on each row. 
            hit_id =  row['hit_id']       # this value is the hit number .     
            #Then I compute the value 
            ext_func_value = ext_func(r0, R, r)
            tmp.append( [hit, ext_fun] )
        
        df_candidates = pd.DataFrame(tmp_candidates, columns=["hit_id", "ext_fun"])
        # Finally 
        # I will find the minimum value of the  extrapolation function
        index = df_candidates['ext_fun'].idxmin()
        new_hit_id = df_candidates['hit_id'][index]
        new_track = [new_hit_id] + track             
        new_tracks.append(new_track)
        #
    return new_tracks # new

In [125]:
def GENERAL_ALGORITHM():
    fraction = 0.004 
    dphi     = 0.01      
    
    df_original = reading_data(fraction)  # where data is unmodified.
    df_search   = df_original             # where I am searching 
    tracks = []                           # [[1,24, 5], [7,6,4] ,[346,7,32,], ... ]
    
    #SEPARATION BY MODULE
    modules = sortbyphi(df_original) 
    df = df_original                      # df is global variable 

    m = len(modules)                         # number of modules
    n = len(df_original['hit_id'].unique())  # number of hits                                             
       
    #ITERATION OVER MODULES ( ):
    for M_i in range(len(modules)-1, 2, -1):
        M_i = M_i - 1
        #1th STEP: module #m-1 = M_i
        left_mod  =  modules[M_i - 1]
        mod       =  modules[M_i    ]
        right_mod =  modules[M_i + 1]                                                                      
        
        mod       = findcandidatewindows(left_mod, mod, right_mod, dphi)
        
        print('+++before trackseeding++++')
        print(tracks)
        print('+++++++')
        
        tracks    = trackseeding(mod, left_mod, right_mod)
        
        print('++++after trackseeding+++')
        print(tracks)
        print('+++++++')
        
        work_module = modules[M_i - 2]
        tracks    = track_forwarding(tracks, work_module, dphi)          # return new tracks

    return tracks

In [126]:
def weaktrackfilter():
    '''Description: '''
    #               
    #                
    #

In [127]:
def forming_track():
    '''
    '''

# Doing Proofs 

In [128]:
fraction = 0.004  
dphi     = np.pi/3                                                                                           

df_original = reading_data(fraction)  # where data is unmodified.
df_search   = df_original             # where I am searching 
df = df_original
tracks = []                           #[[1,24, 5], [7,6,4] ,[346,7,32,], ... ]



#SEPARATION BY MODULE
modules = sortbyphi(df_original)      #                               
df = df_original                      # 

    
m = len(modules)                # number of modules
n = len(df_original['hit_id'].unique())  # number of hits    

GENERAL_ALGORITHM()

            x         y    z        t        dx        dy   dz    dt  \
10900 -14.247  0.461835 -277  696.479  0.015877  0.015877  0.1  50.0   

       particle_id  hit_id       phi  
10900           19   13506 -1.538391  
             x        y    z        t        dx        dy   dz    dt  \
58173  11.1987 -1.47207 -277  695.993  0.015877  0.015877  0.1  50.0   

       particle_id  hit_id       phi  
58173           23   72913 -1.440096  
           x       y    z        t        dx        dy   dz    dt  \
8271 -17.182  4.9501 -277  697.229  0.015877  0.015877  0.1  50.0   

      particle_id  hit_id       phi  
8271           34   10256 -1.290294  
Empty DataFrame
Columns: [x, y, z, t, dx, dy, dz, dt, particle_id, hit_id, phi]
Index: []
            x        y    z        t        dx        dy   dz    dt  \
7074 -18.8091  26.5358 -277  701.965  0.057735  0.057735  0.1  50.0   

      particle_id  hit_id       phi  
7074           29    8763 -0.616621  
             x        y    z  

/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/sit

------------------------The values of L R---are:-----------------------
1.1263856320263188 1.126408115182736
--------------------------------------------------
------------------------The values of hits left and right ---are:-----------------------
2229 15412 7528
--------------------------------------------------
*************triplet************************/n********************************
[2229, 15412, 7528]
*************************************/n********************************
[0.9451902179132503, 0.9451902179132503, 0.9451902179132503]
[-277, -277, -277]
[][][][][][][]][PHIDATA
[0.9451902179132503, 0.9451902179132503, 0.9451902179132503]
[][][][][][][]][PHIDATA
[-277, -277, -277]
[][][][][][][]][][][][][]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
[2229, 15412, 7528, 4908544194.1041]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
------------------------The values of L R---are:-----------------------
1.1263856320263188 1.1264628128775076
------------------------------

/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.


1.1263856320263188 1.2356566523876031
--------------------------------------------------
------------------------The values of hits left and right ---are:-----------------------
2229 15412 71028
--------------------------------------------------
*************triplet************************/n********************************
[2229, 15412, 71028]
*************************************/n********************************
[0.9451902179132503, 0.9451902179132503, 0.9451902179132503]
[-277, -277, -277]
[][][][][][][]][PHIDATA
[0.9451902179132503, 0.9451902179132503, 0.9451902179132503]
[][][][][][][]][PHIDATA
[-277, -277, -277]
[][][][][][][]][][][][][]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
[2229, 15412, 71028, 4908544194.1041]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
------------------------The values of L R---are:-----------------------
1.1263856320263188 1.5014293202178355
--------------------------------------------------
------------------------The values of hits lef

...

/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.

------------------------The values of L R---are:-----------------------
1.2356798332847416 1.2355676008853635
--------------------------------------------------
------------------------The values of hits left and right ---are:-----------------------
74832 90593 59328
--------------------------------------------------
*************triplet************************/n********************************
[74832, 90593, 59328]
*************************************/n********************************
[0.9451902179132503, 0.9451902179132503, 0.9451902179132503]
[-277, -277, -277]
[][][][][][][]][PHIDATA
[0.9451902179132503, 0.9451902179132503, 0.9451902179132503]
[][][][][][][]][PHIDATA
[-277, -277, -277]
[][][][][][][]][][][][][]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
[74832, 90593, 59328, 4908544194.1041]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
------------------------The values of L R---are:-----------------------
1.2356798332847416 1.2356010957440284
-----------------------

/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.ch/work/p/ppayemam/miniconda/envs/ramp_velo_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:63: RankWarning: Polyfit may be poorly conditioned
/afs/cern.

[][][][][][][]][][][][][]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
[74832, 90593, 71254, 4908544194.1041]
PRINTING PRINTING PRINTING PRINTINGI PRINTING LIST
------------------------The values of L R---are:-----------------------
1.2356798332847416 1.2356676026517857
--------------------------------------------------
------------------------The values of hits left and right ---are:-----------------------
74832 90593 73067
--------------------------------------------------
*************triplet************************/n********************************
[74832, 90593, 73067]
*************************************/n********************************


KeyboardInterrupt: 

In [ ]:
I do not understand what is the error

So at first time 
I am trying to develope a new strategy to complete the homeworks. 

First at all 
The strategy will be the next 

Clustering 

So, the next informaiton that I have recived is 

DO WE NNED TIME INFORMATION ON VELO HITS?

IF SO, HOW PRECISE?

HOW IMPORTANT IS TIME PRECISION?

COMPARED TO PIXEL SIZE 

5 VELO OPTIONS 

DIFFERENT PIXEL SIZE TIME PRECISION

# How I can extract the information of triplets to extrapolate to the working module. 

In [ ]:
triplet = (164, 199, 192)
df.head()                  # It is a dataframe.

In [ ]:
phi_data = [ df.query(f'hit_id == {827}')['phi'] for hit in triplet ]
z_data   = [ df.query(f'hit_id == {827}')['z'] for hit in triplet ]
phi_data = [ hit.values[0] for hit in phi_data ]
z_data   = [ hit.values[0] for hit in z_data ]

print(phi_data)
print(z_data)

In [ ]:
phi_data.values[0]

In [ ]:
df['hit_id'].unique

## Since the value of the dataframe is unique, so, it is possible to access to the information

In [ ]:
phi_data = [ df.query(f'hit_id == {827}')['phi'] for hit in triplet ]
z_data   = [ df.query(f'hit_id == {827}')['z'] for hit in triplet ]
phi_data = [ hit.values[0] for hit in phi_data ]
z_data   = [ hit.values[0] for hit in z_data ]

print(phi_data)
print(z_data)